In [1]:
import pennylane as qml
from pennylane import numpy as np
import qiskit
import sys

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


## Problem description
- Implement a circuit that returns $|01\rangle$ and $|10\rangle$ with equal probability.
- Use only `CNOT`, `RX`, and `RY`.
- Find parameters with gradient descent.
- Simulations must be done with sampling and noise.

### Intuition
The circuit that returns $|00\rangle$ and $|10\rangle$ with equal probability is one that generates a state vector of the form

\begin{equation}
|\Phi\rangle = e^{i \theta} \frac{1}{\sqrt{2}} [|01\rangle + e^{i \phi}|10\rangle].
\end{equation}

The global phase, $e^{i \theta}$, is physically irrelevant. The relative phase $e^{i \phi}$ will not effect the probabilities of measurement outcomes on this state, but different relative phase factors correspond to quantum states which *are* physically different.
<br><br><br>
To keep it simple, let's consider the state 
\begin{equation}
|\Psi\rangle = \frac{1}{\sqrt{2}} [|01\rangle + |10\rangle].
\end{equation}

**What is the non-parametric circuit that generates this state vector?**

Well, $|\psi\rangle$ looks kind of like the simplest Bell state $|\beta_{00}\rangle = \frac{1}{\sqrt{2}} [|00\rangle + |11\rangle]$, in fact $|\Psi\rangle$ is itself a Bell state. The state, $|\beta_{00}\rangle$, says that qubit one and qubit two will be in the same compuational basis state upon measurement. We want the opposite of that: if one qubit is in the $|0\rangle$ state, the other ought to be in the $|1\rangle$ state, and vice versa. Since, we want to flip the results, it sounds like we might need an `X` gate.
<br><br>
This intuition proves correct, as a simple way to implement the circuit that acheives our goals is to modify the familiar circuit for $|\beta_{00}\rangle$ by inserting an additional `X` gate on the second qubit.

In [2]:
qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.x(1)
qc.cx(0, 1)
# qc.x(1) # alternative X-gate position

backend = qiskit.Aer.get_backend("statevector_simulator")
psi = qiskit.execute(qc, backend).result().get_counts()

print(qc)
print(psi)

     ┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
{'01': 0.5, '10': 0.5}


**Now with our gate set**

Our gate set is {`CNOT`, `RX`, `RY`}. So what is a corresponding parameteric circuit?

Well, if we try to match gates from the circuit above to our gate set, obviously we have `CNOT`. Further, $RY(\frac{\pi}{2}) = H$. 

We can't exactly get the `X` with any paramaterization of a single `RX` or `RY`. However, we can get the same effect on the $|0\rangle$ state as 
\begin{equation}
RY(\pi)|0\rangle = |1\rangle, \\
RX(\pi)|0\rangle = |1\rangle.
\end{equation}

I'll choose to use `RY`. This way, we keep the coefficients of our state vector purely real. 

In [3]:
qc = qiskit.QuantumCircuit(2)
qc.ry(np.pi/2, 0)
qc.ry(np.pi, 1)
qc.cx(0, 1)

backend = qiskit.Aer.get_backend("statevector_simulator")
psi = qiskit.execute(qc, backend).result().get_counts()

print(qc)
print(psi)

     ┌──────────┐     
q_0: ┤ RY(pi/2) ├──■──
     └┬────────┬┘┌─┴─┐
q_1: ─┤ RY(pi) ├─┤ X ├
      └────────┘ └───┘
{'01': 0.5, '10': 0.5}


### Some analytic calculations
We've proven that the circuit above is expressive enough to generate the state $|\Psi\rangle$.

We can represent our parameterized circuit as

\begin{equation}
CX_{0,1}RY_1(\theta_1)RY_0(\theta_0)|00\rangle = |\psi\rangle.
\end{equation}

With arbitrary prametizeration, doing the matrix calculations gives us

\begin{equation}
|\psi\rangle = cos(\theta_0)cos(\theta_1)|00\rangle + cos(\theta_0)sin(\theta_1)|01\rangle + sin(\theta_0)sin(\theta_1)|10\rangle + sin(\theta_0)cos(\theta_1)|11\rangle.
\end{equation}
<br><br>
**How do we know if our circuit gives a good answer?**

Since the problem requires that we sample we'll only have access the counts, and from the counts we can compute the probabilities of various measurement results. 

We know the desired probabilities -- in vector form they are $$ \vec{y} = [0, 0.5, 0.5, 0]^T.$$

The vector of probabilites given by $|\psi\rangle$ is 

\begin{equation}
\vec{p} = [|cos(\theta_0)cos(\theta_1)|^2, |cos(\theta_0)sin(\theta_1)|^2, |sin(\theta_0)sin(\theta_1)|^2, |sin(\theta_0)cos(\theta_1)|^2].
\end{equation}

But, we can simplify a bit. Notice that our coefficients are all real. For any $x \in \mathbb{R}$, we have $|x| = \sqrt{x^2}$. With this identity we can simplify our probability vector,

\begin{equation}
\vec{p} = [(cos(\theta_0)cos(\theta_1))^2, (cos(\theta_0)sin(\theta_1))^2, (sin(\theta_0)sin(\theta_1))^2, (sin(\theta_0)cos(\theta_1))^2].
\end{equation}

**Loss function**

With, $\vec{y}$ and $\vec{p}$ we can construct a Mean Squared Error loss function, $\mathcal{L}$.

\begin{equation}
\mathcal{L} = \sum_i = (\vec{p}_i - \vec{y}_i)^2
\end{equation}

In [47]:
y = np.array([0, 0.5, 0.5, 0])

def loss(p):
    return np.array([np.sum((p - y)**2)])

**The gradients and the update rule**

To know how to update the parameters $\theta_0$ and $\theta_1$, we need to take the gradient of the loss function with respect to the parameters. 

\begin{equation}
\nabla \mathcal{L} = [\frac{\mathcal{L}}{\theta_0}, \frac{\mathcal{L}}{\theta_1}]^T
\end{equation}

The update rule is then
\begin{equation}
\vec{\theta}^{(t+1)} = \vec{\theta}^{(t)} - \alpha \nabla \mathcal{L}
\end{equation}

In [48]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
params = [0, 0]

for i in range(steps):
    # update the circuit parameters
    params = opt.step(loss, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

TypeError: Can't differentiate w.r.t. type <class 'int'>

### Penny lane

In [35]:
dev = qml.device('default.qubit', wires=2, shots=1000, analytic=False)

@qml.qnode(dev)
def circuit(theta_vector):
    qml.RZ(theta_vector[0], wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(theta_vector[1], wires=1)
    return qml.probs([1])

result = circuit([0.5, 0.5])

In [36]:
for _ in range(10):
    print(circuit([0.5, 0.5]))

[0.931 0.069]
[0.942 0.058]
[0.942 0.058]
[0.944 0.056]
[0.944 0.056]
[0.946 0.054]
[0.934 0.066]
[0.939 0.061]
[0.941 0.059]
[0.936 0.064]


### Loss function

In [ ]:
lss = 

### Qiskit

This is a possible circuit that outputs the desired state.

In [41]:
qc = qiskit.QuantumCircuit(2)
qc.ry(np.pi/2, 0)
qc.ry(np.pi, 1)
qc.cx(0, 1)

backend = qiskit.Aer.get_backend("statevector_simulator")
psi = qiskit.execute(qc, backend).result().get_counts()

print(qc)
print(psi)

     ┌──────────┐     
q_0: ┤ RY(pi/2) ├──■──
     └┬────────┬┘┌─┴─┐
q_1: ─┤ RY(pi) ├─┤ X ├
      └────────┘ └───┘
{'01': 0.5, '10': 0.5}


past

In [9]:

#Changing the simulator 
backend = qiskit.Aer.get_backend('unitary_simulator')

result = qiskit.execute(qc, backend).result()

print(result.get_unitary(qc, decimals=1))

[[ 4.32978028e-17+0.j -4.32978028e-17+0.j -7.07106781e-01+0.j
   7.07106781e-01+0.j]
 [ 7.07106781e-01+0.j  7.07106781e-01+0.j  4.32978028e-17+0.j
   4.32978028e-17+0.j]
 [ 7.07106781e-01+0.j -7.07106781e-01+0.j  4.32978028e-17+0.j
  -4.32978028e-17+0.j]
 [ 4.32978028e-17+0.j  4.32978028e-17+0.j -7.07106781e-01+0.j
  -7.07106781e-01+0.j]]


This is hadamard

In [10]:
qc = qiskit.QuantumCircuit(1)
qc.ry(np.pi/2, 0)

backend = qiskit.Aer.get_backend("statevector_simulator")
psi = qiskit.execute(qc, backend).result().get_counts()

print(qc)
print(psi)

     ┌──────────┐
q_0: ┤ RY(pi/2) ├
     └──────────┘
{'0': 0.5, '1': 0.5}


In [11]:
RY = lambda theta : qiskit.circuit.library.standard_gates.RYGate(theta)
RY(np.pi/2).to_matrix()

array([[ 0.70710678+0.j, -0.70710678+0.j],
       [ 0.70710678+0.j,  0.70710678+0.j]])

This is X with phase

In [12]:
RX = lambda theta : qiskit.circuit.library.standard_gates.RXGate(theta)
RY(np.pi/4).to_matrix() @ RX(np.pi/2).to_matrix()

array([[ 0.65328148+0.27059805j, -0.27059805-0.65328148j],
       [ 0.27059805-0.65328148j,  0.65328148-0.27059805j]])

In [13]:
H = qiskit.circuit.library.standard_gates.HGate()

In [14]:
RX(np.pi/2).to_matrix()

array([[0.70710678+0.j        , 0.        -0.70710678j],
       [0.        -0.70710678j, 0.70710678+0.j        ]])

In [15]:
qc = qiskit.QuantumCircuit(2)
qc.ry(np.pi/2, 0)
qc.ry(np.pi, 1)
qc.cx(0, 1)
# qc.x(1) # alternative X-gate position

backend = qiskit.Aer.get_backend("statevector_simulator")
psi = qiskit.execute(qc, backend).result().get_statevector()

print(qc)
print(psi.round(2))

     ┌──────────┐     
q_0: ┤ RY(pi/2) ├──■──
     └┬────────┬┘┌─┴─┐
q_1: ─┤ RY(pi) ├─┤ X ├
      └────────┘ └───┘
[0.  +0.j 0.71+0.j 0.71+0.j 0.  +0.j]
